In [1]:
# Librerías necesarias
import pandas as pd
import re
import requests

In [31]:
df = pd.read_csv("ficheros/areas_ac.csv")
df.head()

,WKT,NOMBRE
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...
2,POINT (0.35921 42.90715 0.0),Arreau Ã¡rea AC municipal (2 eurosAgua:sivaci...
3,POINT (-0.41007 38.39479 0.0),Camper Area Campello Beach (14eur/nocheAgua:...
4,POINT (-8.44443 43.37179 0.0),A CoruÃ±a Gratuito: Si Pernocta: Si WC: Si Ag...


In [3]:

df.isnull().sum()

WKT       0
NOMBRE    0
dtype: int64

In [4]:
df.dtypes

WKT       object
NOMBRE    object
dtype: object

### EXTRAER LA INFORMACIÓN LAS COLUMNAS
---

In [32]:
# Función para sacar las coordenadas de la columna WKT
def sacar_latitud(col):
    patron = "-?\d+\.\d+" 
    return re.findall(patron, col)[1]

def sacar_longitud(col):
    patron = "-?\d+\.\d+" 
    return re.findall(patron, col)[0]

In [33]:
df["latitude"]= df["WKT"].apply(sacar_latitud)

In [34]:
df["longitude"] = df["WKT"].apply(sacar_longitud)

In [35]:
df.head(2)

,WKT,NOMBRE,latitude,longitude
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696


In [36]:
# Función para buscar si es de pago o no
def precio(col):
    
    patron1 = "[G|g]ratuito:*\s*[N|n]o"
    patron2 = "[G|g]ratuito"
    patron3 = ".*\d+"
    if re.findall(patron1,col):             
        return "Paying"
    elif re.findall(patron2,col):             
        return "Free"
    elif re.findall(patron3,col):             
        return "Paying"
    else:
        return "Free"

In [37]:
df["price"] = df["NOMBRE"].apply(precio)

In [38]:
df.head(2)

,WKT,NOMBRE,latitude,longitude,price
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Free
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Free


In [39]:
df.price.value_counts()

Free      552
Paying    361
Name: price, dtype: int64

In [40]:
# Función para saber si disponen de servicio de aguas o no
def buscar_agua(col):
    patron = "[A|a]gua:*\s*[N|n]o*"
    patron2 = "[A|a]gua:*\s*[S|s]*i*"
    if re.findall(patron, col):
        return "no"
    elif re.findall(patron2, col):
        return "yes"
    else:
        return "unknown"

In [41]:
df["water"] = df["NOMBRE"].apply(buscar_agua)

In [42]:
df.head(2)

,WKT,NOMBRE,latitude,longitude,price,water
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Free,yes
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Free,yes


In [43]:
df["water"].value_counts()

yes        885
no          19
unknown      9
Name: water, dtype: int64

In [46]:
# Función para saber si tienen wc
def buscar_wc(col):
    patron = "[W|w][C|c]:*\s*[N|n]o*"
    patron2 = "[W|w][C|c]:*\s*[S|s]*i*"
    if re.findall(patron, col):
        return "no"
    elif re.findall(patron2, col):
        return "yes"
    else:
        return "unknown"

In [47]:
df["wc"] = df["NOMBRE"].apply(buscar_wc)

In [48]:
df.head(2)

,WKT,NOMBRE,latitude,longitude,price,water,wc
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Free,yes,yes
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Free,yes,yes


In [49]:
df.wc.value_counts()

no         511
yes        363
unknown     39
Name: wc, dtype: int64

In [50]:
# Función para saber si le han etiquetado el servicio de vaciado
def vaciado(col):
    patron = "[V|v]aciado:*\s*[N|n]o*"
    patron2 = "[V|v]aciado:*\s*[S|s]*i*"
    if re.findall(patron, col):
        return "no"
    elif re.findall(patron2, col):
        return "yes"
    else:
        return "unknown"

In [52]:
df["grey_water"] = df["NOMBRE"].apply(vaciado)

In [53]:
df.head(2)

,WKT,NOMBRE,latitude,longitude,price,water,wc,grey_water
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Free,yes,yes,yes
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Free,yes,yes,unknown


In [54]:
df.grey_water.value_counts()

yes        629
unknown    278
no           6
Name: grey_water, dtype: int64

In [55]:
# Después de extraer la información, elimino la columna WKT
df.drop(columns=["WKT", "NOMBRE"], axis=1, inplace= True)

### API PARA EXTRAER LA LOCALIDAD, PROVINCIA Y PAIS
--- 

In [56]:
def localidad(lat, lon):
    
    # hacemos la llamada  a la API
    url = f'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}'

    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason
   
    respuesta = response.json()
    direccion = respuesta["address"]
    lista = direccion.keys()
    
    # Dependiendo del país y del tamaño, vienen con diferente denominación
    if "town" in lista:        
        return direccion["town"]
    elif "city" in lista:
        return direccion["city"]
    elif "village" in lista:
        return direccion["village"]
    elif "municipality" in lista:
        return direccion["municipality"]
    else:
        return "unknown"

In [58]:
df["town"] = df.apply(lambda ciudad: localidad(ciudad["latitude"], ciudad["longitude"]), axis= 1)

In [59]:
df.head()

,latitude,longitude,price,water,wc,grey_water,town
0,39.06956,-0.4098,Free,yes,yes,yes,Carcaixent
1,42.64159,-8.89696,Free,yes,yes,unknown,Boiro
2,42.90715,0.35921,Paying,yes,no,yes,Arreau
3,38.39479,-0.41007,Paying,yes,yes,unknown,el Campello
4,43.37179,-8.44443,Free,yes,yes,unknown,A Coruña


In [60]:
df["town"].value_counts()

Encamp                  3
Thann                   2
Campan                  2
Roma                    2
Ljubljana               2
                       ..
Jumièges                1
Kalmar                  1
Karlskrona              1
Kaysersberg-Vignoble    1
Cúllar                  1
Name: town, Length: 887, dtype: int64

In [61]:
def region(lat, lon):
    
    # hacemos la llamada  a la API
    url = f'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}'

    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason
   
    respuesta = response.json()
    direccion = respuesta["address"]
    lista = direccion.keys()
    
    if "state" in lista:
        return direccion["state"]
    elif "county" in lista:
        return direccion["county"]
    else:
        return direccion["country"]

In [62]:
df["region"] = df.apply(lambda pepe: region(pepe["latitude"], pepe["longitude"]), axis= 1)

In [63]:
df.head(2)

,latitude,longitude,price,water,wc,grey_water,town,region
0,39.06956,-0.4098,Free,yes,yes,yes,Carcaixent,Comunitat Valenciana
1,42.64159,-8.89696,Free,yes,yes,unknown,Boiro,Galicia


In [64]:
df["region"].value_counts()

Nouvelle-Aquitaine     100
Occitanie               66
Catalunya               64
Galicia                 58
Andalucía               38
                      ... 
Puglia                   1
Norge                    1
England                  1
Comunidad de Madrid      1
San Marino               1
Name: region, Length: 111, dtype: int64

In [65]:
def pais(lat, lon):
    
    # hacemos la llamada  a la API
    url = f'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}'

    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason
   
    respuesta = response.json()
    direccion = respuesta["address"]
    lista = direccion.keys()
    
    return direccion["country"]

In [66]:
df["country"] = df.apply(lambda pepe: pais(pepe["latitude"], pepe["longitude"]), axis= 1)

In [67]:
df.head(2)

,latitude,longitude,price,water,wc,grey_water,town,region,country
0,39.06956,-0.4098,Free,yes,yes,yes,Carcaixent,Comunitat Valenciana,España
1,42.64159,-8.89696,Free,yes,yes,unknown,Boiro,Galicia,España


In [68]:
df["country"].value_counts()

España                            347
France                            343
Deutschland                        62
Italia                             59
Portugal                           37
Slovenija                          12
Norge                              10
Sverige                             8
Andorra                             6
Nederland                           5
Danmark                             5
België / Belgique / Belgien         5
Schweiz/Suisse/Svizzera/Svizra      2
Österreich                          2
United Kingdom                      2
Hrvatska                            2
Australia                           1
Suomi / Finland                     1
Lëtzebuerg                          1
Polska                              1
Slovensko                           1
San Marino                          1
Name: country, dtype: int64

In [69]:
df.head(2)

,latitude,longitude,price,water,wc,grey_water,town,region,country
0,39.06956,-0.4098,Free,yes,yes,yes,Carcaixent,Comunitat Valenciana,España
1,42.64159,-8.89696,Free,yes,yes,unknown,Boiro,Galicia,España


In [70]:
df = df.reindex(columns=["latitude", "longitude", "town", "region", "country", "price", "water", "wc", "grey_water"])
df.head(2)

,latitude,longitude,town,region,country,price,water,wc,grey_water
0,39.06956,-0.4098,Carcaixent,Comunitat Valenciana,España,Free,yes,yes,yes
1,42.64159,-8.89696,Boiro,Galicia,España,Free,yes,yes,unknown


In [71]:
df.to_csv("ficheros/areas_limpio.csv")